In [1]:
# Initializing Spark
with open("setupPySpark.py", "r") as setup_file:
    exec(setup_file.read())


In [2]:
from pyspark.sql.session import SparkSession

spark = SparkSession(sc)

In [3]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

In [4]:
from util import read_file, read_folder
from pandas import Series

In [5]:
in_file = open("in_file.cfg").read()
in_file = in_file.split("\n")
patterns_file = in_file[5]
file_folder = in_file[4]
label = in_file[3]
column = in_file[2]
in_type = in_file[1]
in_file = in_file[0]
if file_folder == "file":
    strings = read_file(in_file, in_type = in_type)
    if in_type == "text":
        strings = tokenize_sentence_nltk(strings)
        strings = DataFrame(strings)[0]
    elif in_type == "html":
        timestamp = strings[2]
        meta_data = strings[1]
        strings = strings[0]
        strings[label] = meta_data["Comment"]
        labels = strings[label]
        strings = strings[column]
    else:
        if label in strings.columns:
            labels = strings[label]
        strings = strings[column]
else:
    strings = read_folder(in_file, in_type = in_type)
    patterns = Series([".*" + x + ".*" for x in open(patterns_file, 'r').readlines()])

In [6]:
strings['conversation'] = strings['conversation'].apply(lambda x: ". ".join(x["Message"]))
sentenceDataFrame = spark.createDataFrame(strings)

In [7]:
from util_spark import remove_stopwords_spark
from tokenization_spark import tokenize_sentence_nltk_spark

In [8]:
sentenceDataFrame = tokenize_sentence_nltk_spark(df = sentenceDataFrame, in_col = "conversation")

In [9]:
from util_spark import detect_language_spark

In [10]:
sentenceDataFrame = detect_language_spark(df = sentenceDataFrame, in_col = "conversation")

In [11]:
sentenceDataFrame.select(['conversation']).show(1, truncate = False)

+------------+
|conversation|
+------------+
|en          |
+------------+
only showing top 1 row

